# NGS Application-Variant Calling

- author: "Kwon DoHyung"
- toc: true 
- comments: true
- categories: [NGS, bioinformatics]
- image: images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled3.png
- permalink: /NGS--Variant-Calling/

# Introduction
NGS 기계를 통해 얻은 raw data는 기본적으로 리눅스 시스템에서 처리된다. 이때 사용되는 툴킷은 대표적으로 broad institute에서 만든 <a href="https://gatk.broadinstitute.org/hc/en-us/sections/360007226651-Best-Practices-Workflows">GATK</a>가 있다. <br>
이번 노트에서는 NGS의 여러 응용(Application) 중 Variant calling에 대해 다룬다. 본 자료는 cornell university의 workshop자료를 기반으로 한다.

# Variant Calling

## 정의

variant는 <u>유전적변이</u>를 의미한다. (네이버 사전) <br>
variant call은 genome 혹은 transcriptome에서 reference genome과 뉴클레오타이드 서열에 차이가 있는지를 살펴보는 것을 말한다. 즉, reference genome의 시퀀스와 대상 genome 시퀀스 상의 변화를 찾아낸다. 즉, SNP를 찾아낸다.<br>
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/SNP.jpg)

SNP란 인구 집단에서 1% 이상의 빈도로 존재하는 2개 이상의 대립 염기서열이 발생하는 위치를 말한다. 각 샘플의 Read data (D)로부터 샘플의 Genotype (T)을 알아내려면 Genotype이 나올 확률을 Bayesian rule을 이용하여 계산하게 된다. SNP 외에 염기서열 상의 insertion, deletion 등의 mutation도 찾아낸다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled3.png)

Individual 1은 시퀀스를 하려는 특정한 사람의 DNA. 혹은 정상세포에서 얻은 DNA, Individual 2는 또 다른 사람의 DNA, Individual 3는 암 환자에게서 추출한 세포에서 얻은 DNA라고 가정해보자. 이렇게 서로 다른 세포에서 얻은 두 genome의 차이를 살펴, 암을 유발하는 사이트를 알아내려는 것이 목적이다. 보통 휴먼지놈은 30억 bp다. 이를 통째로 시퀀싱 하지 못하므로, 잘게 잘라낸다. 이를 통해 작은 read들을 NGS를 통해 얻어내고 그 결과들을 이어붙여 원래의 whole genome 시퀀스를 확보한다. 그 후에 상호 간의 시퀀스를 비교한다. 이러한 과정들은 사람이 직접 할 수 없으므로 프로그램에 맡긴다.
NGS 기계에서 최종적으로 나오는 output은 FASTQ 포맷의 파일이며 그 예는 다음과 같다. FASTQ에 대해서는 후반부에 다룬다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled4.png)

이런 류의 파일을 모은 후, FASTQ 파일에 있는 데이터들에 대한 align을 수행하게 된다. 중첩된 것들은 따라가면서 전체 시퀀스를 어우르게 된다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled5.png)

reference genome sequence는 휴먼 지놈 프로젝트를 통해 밝혀진, 이미 알려진 genome sequence다. 사람에 대한 reference sequence는 이미 존재하므로 이를 기반으로 FASTQ 파일로부터 align된 시퀀스들을 reference sequence로 맵핑한다. 이 과정에서 GATK가 사용된다.

# GATK

NGS 기계를 통해 얻은 raw data는 GATK 툴킷으로 주로 처리된다. GATK 외에 SAMtools, glfTools, Atlas2가 있다. GATK는 Bayesian 모델을 이용하고, 알고리즘은 MapReduce를 이용하여 분산 처리가 가능하다는 장점이 있다. GATK는 java program으로 짜여져 있다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Comparison_of_Variant_Callers.png)
휴먼 지놈 프로젝트는 2000년대에 완성되었다. 그 후, 1000 명에 대한 genome sequencing 즉, 1000 genome project가 진행되었고 현재는 종료된 상태다. 이때 발생한 패키지가 GATK다. 자바로 짜여진 프로그램이며, command-line의 툴이기 때문에 명령창에서 명령어를 입력해야 사용할 수 있다. 추가적으로 명령어 뒤의 옵션에 따라 다음의 일을 수행할 수 있다.
- NGS data processing
- variant discovery
- variant filtering
- genotyping
- evaluation(평가 기능)
- parallel processing on HPC clusters(병렬 처리 기능)
    - 병렬 처리
        - 일반적인 소프트웨어는 하나의 CPU에 하나의 코어를 점유한다. 즉, 싱글 코어를 사용한다. 그러나 병렬 처리를 한다는 것은 여러 코어(또는 여러 쓰레드)를 점유하여 빠르게 처리하는 것을 말한다. 즉, 계산할 데이터를 나누어 각 CPU 코어에 계산을 맡긴다. 이때 CPU 내의 여러 코어를 사용할 수도 있고, 쓰레드를 사용할 수도 있다. 
        - GATK 실행 시, 명령 옵션을 통해 쓰레드 수를 지정할 수 있다. 
        - 그러나 단순히 쓰레드 수를 늘린다고 속도가 높아지는 것은 아니다. 즉, 쓰레드 수에 따라 선형적인 성능을 보장하지 않는다. 이는 오버헤드가 발생하기 때문이다. 일을 나누어주는 것 자체가 일이며, 분산시킨 작업의 결과를 취합하는 것 또한 CPU에겐 부담이다. 따라서 여러 코어, 여러 쓰레드를 쓰는 것이 선형적인 성능을 보장하지 않는다. 일반적으로 8~10개 정도가 적당하다.
    
GATK에 대한 BEST practice는 다음의 사이트에서 참고할 수 있다; [https://gatk.broadinstitute.org/hc/en-us](https://gatk.broadinstitute.org/hc/en-us)

# NGS data Analysis Pipeline

NGS 데이터를 처리하기 위한 일련의 과정을 파이프라인으로 정의할 수 있다. 각 과정에서는 서로 다른 소프트웨어가 쓰일 수 있으며 어떤 소프트웨어의 output이 검증 과정을 거친 후 다른 소프트웨어의 input이 된다. 즉, input으로 raw data를 주면 자동으로 다음 단계들을 알아서 실행하여 최종적으로 원하는 output이 나오도록 설계하는 것을 파이프라인이라고 한다. 일련의 작업들을 알아서 수행하는 매크로와도 같은 개념이다. 이렇게 파이프라인 만드는 것이 BI 회사들이 주로 하는 일이다. 파이썬, 쉘 스크립트 등을 이용하여 이러한 파이프라인을 구축할 수 있다.

## A. Data Cleanup

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled6.png)
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/NGS_data_analysis_pipeline.png)
1. 먼저, NGS 기계로부터 raw reads를 받는다. 이 데이터는 fastq 포맷의 파일이다.
2. 그 다음엔 해당 파일 내의 데이터를 전처리한다. 불필요한 부분을 잘라내고, 필요한 부분만을 추려내는 과정이다 (실제 사용 예는 하단에 있음). 이렇게 처리된 데이터의 quality를 검증한 후, BWA 소프트웨어를 이용하여 Reference mapping을 수행한다.
3. Alignment(또는 Mapping)가 완료된 후에는 sorting을 수행한다. 
    - sorting이 필요한 이유는 인덱싱 작업을 하기 위해서다. 데이터베이스 상에서 인덱싱 된 데이터일 경우 빠르게 검색 가능하기 때문이다. 이때 picard 소프트웨어가 쓰인다. 이는 다음 작업인 duplicate를 제거하는 작업을 빠르게 한다.
4. duplicate를 체크하고, 제거한다.
    - 과정 4까지는 Non-GATK 즉, GATK가 쓰이지 않는다. 과정 5, 6은 GATK가 쓰인다.
5. 그 후 indel(insertion-deletion) realignment 처리 후, 
6. base recalibration을 수행
    - 과정 6까지 끝나고 난 output: analysis-ready reads. 
    - 이로써 Variant Discovery를 위한 기본적인 준비는 끝이 났다.
    
#### pair of FASTQ files

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled7.png)
위 그림은 각 샘플에 대한 작업을 의미한다. 모든 과정에서, 샘플은 하나가 아닌 여러 샘플을 동시에 처리하는 경우가 대부분이다. 사람마다 genome은 다른데 reference sequence는 딱 하나다. 즉, 질병에 의한 variaion 뿐만 아니라 사람 간의 variation도 존재한다. 우리의 목적이 무엇이냐에 따라 다르겠으나, (어떠한 genotype이 어떠한 phenotype을 나타내는지에 관심이 있는 경우도 있다.) 주로 질병 치료, 진단 등이 목적이 된다 (돈이 되기 때문). 즉, 정상과 비정상간의 variant calling이 주 목적이다. 이때 사람 간의 variation은 무시해야 하므로, 통계적 유의미성 확보를 위해 굉장히 많은 샘플을 이용해야 한다. 따라서 몇 백~몇 천 명의 샘플이 필요하다. 따라서 각 샘플은 동일한 파이프라인을 거치게 된다. 
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled8.png)

## B. Variant Discovery

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled9.png)
처리한 것들을 묶어서 variant calling을 수행한다. 이를 통해 샘플 간 정상과 비정상간의 SNP를 알 수 있다 (몇 번 크로모솜의 어느 site에서 변이가 일어났는지, indel(insertion, deletion)이 일어났는지 등). 
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled10.png)
Data Cleanup, Variant Discovery 과정까지가 bioinformatics의 영역이라고 할 수 있다. 

## C. Evaluation

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled11.png)
Evaluation 단계는 연구자들 또는 의사들의 영역이라고 할 수 있다. 즉, 결과물로 나온 SNP가 의미가 있는 SNP인지 의미가 없는 SNP인지에 대한 판단을 수행한다. <br>
예를 들어, 특정 역할을 하는 단백질, 이를 테면 kinase(신호 전달 체계의 핵심 역할을 하는 단백질)가 있다고 해보자. 이 단백질에 뮤테이션이 발생하여 SNP로 드러나면 해당 SNP가 단백질의 구조에 어떠한 영향을 얼마나 주는지에 대해 판단해야 한다. 즉, 뉴클레오타이드 A가 T로 변화되었고, 이로 인해 아미노산이 Lue에서 Glu으로 바뀌었다면, 해당 단백질의 activity에 영향이 있는가, 없는가를 판단해야 한다. 이를 통해 "해당 SNP에 의한 특정 단백질의 활성화로 인해 암이 발생할 확률이 있다."는 식으로 시나리오를 짜볼 수 있겠다.<br>
그러나, SNP는 한두 개 정도가 나오는 게 아니다. 정상 세포와 비정상(암) 세포 간 SNP, indel은 굉장히 많이 발생한다. 이로 인해 false positive들이 상당히 많이 발생할 수 있다. 이 false positive를 가려내는 것이 관건이다. 또한, SNP를 발견하여 단백질의 activity를 알아보기 위해서는 실제로 mutation을 도입한 후 클로닝하여 단백질 발현을 통해 activity 테스트를 해야한다. SNP가 한, 두 개 발생하는 것이 아니기 때문에 이 모든 경우를 전부 테스트 할 수는 없다는 한계가 있다.<br>
따라서, 이 부분에 인공지능이 개입할 여지가 있으며 실제로 인공지능을 도입하려는 시도들이 많이 이루어지는 추세다. 인공지능을 통해 정상 세포가 암 세포로 전이될 확률 등을 알아낼 수 있다면, 의사들과 연구자들의 빠른 의사결정에 도움을 줄 수 있을 것이다. 

# FASTQ

FASTQ는 NGS 기계의 output으로 주어지는 데이터 파일이다. 이 파일의 생김새는 아래와 같다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled12.png)
FASTA 포맷과 다른 점은 `>`기호 대신 `@`기호가 쓰인다는 점, `:`으로 필드가 나누어지는 점, 해당 시퀀스에 대한 인포메이션의 형식(FASTQ는 몇 번째 read고 어떤 플랫폼으로 시퀀싱 된 것인지에 대한 정보를 담고 있다.), 그리고 단 네 줄로만 이루어진다는 점이다. 또한, FASTA 포맷은 첫번째 줄의 시퀀스에 대한 설명 이후 뉴클레오타이드 또는 아미노산 서열만이 나열되는 데에 비해 FASTQ 포맷은 두 번째 줄에 실제 NGS를 통해 나온 서열이 위치하며, 네번째 줄에 정확도를 나타내기 위한 데이터가 나열된다. `+`는 optional이며, 부가적인 다른 인포메이션을 적을 수 있다. 

## accuracy

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled13.png)
`A`라고 시퀀싱 된 것을 얼마나 신뢰할 수 있을까? 네 번째 줄을 보면 `C`로 표기된 것을 볼 수 있다. 이는 정확도를 나타내는 지표이다. 네 번째 줄의 각 알파벳을 아스키 코드로 바꾼 후 `-33`을 더하면 phred quality score가 나오게 된다. 
- 아스키 코드
    - American Standard Code Information Interchange
    - 영어 알파벳 a-z, 외 특수 기호들에 대하여 컴퓨터가 인식 가능하도록 숫자로 바뀐, 코드화 된 것을 말함
    - A: 65, B: 66, C: 67, ... 
    
네 번째 줄의 `B`를 예로 들자. `B`의 아스키 코드는 `66`이다. phred quality score는 `-33`을 한 값인 -33. 이때 quality score가 30이면 accuracy는 9가 3개인 99.9%를 의미한다 (40이면 99.99%, 50이면 99.999%). -33이므로, 99.9% 이상을 의미한다. 당연하게도 아스키 코드 값이 높아질수록 quality score가 높아진다고 이해할 수 있다.

## paired-end (PE) reads

Illumina NGS 플랫폼에서 만들어진 read는 paired-end (PE) 형태이다. 즉, 하나의 서열을 양 옆에서 읽어들이는 방식이다. 
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled14.png)
이 때의 서열은 다음과 같다.
![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled15.png)
forward(left) 방향으로 읽은 서열엔 `1`, backward(right) 방향으로 읽은 서열엔 `2`가 마킹된 것을 볼 수 있다. 상황과 목적에 따라 `1`로 마킹된 것들만 묶어서 하나의 파일로, `2`로 마킹된 것들만 묶어서 하나의 파일로, 또는 `1`과 `2` 둘다 모두 들어있는 파일로 만들어 활용할 수 있다.

# Data Cleanup: trimmomatic.jar

![](../images/2020-09-11-NGS-Variantcallingc2db69a7f6824b298e5dbc547ae4259f/Untitled16.png)
data cleanup 단계에서 raw data를 cleanup하라는 명령이다. trimmomatic.jar라는 자바 프로그램을 이용하며 역시 커맨드 라인에서 수행된다.

# Reference
- https://biohpc.cornell.edu/lab/doc/Variant_workshop_Part1.pdf
- https://3months.tistory.com/234
- http://www.incodom.kr/GATK/VariantCalling